In [15]:
import pandas as pd
import numpy as np

# Goal: forecast which countries will become top tourism destinations
# by predicting next-year international tourist arrivals.

world = pd.read_csv('world_tourism_economy_data.csv')
arrivals = pd.read_csv('international-tourist-arrivals.csv')

# Normalize/rename arrivals dataset columns
value_col = [c for c in arrivals.columns if c not in ['Entity', 'Code', 'Year']][0]
arrivals = arrivals.rename(
    columns={
        'Entity': 'country_name_arrivals',
        'Code': 'country_code',
        'Year': 'year',
        value_col: 'intl_arrivals',
    }
)

world['country_code'] = world['country_code'].astype(str).str.upper()
arrivals['country_code'] = arrivals['country_code'].astype(str).str.upper()

print('World rows:', world.shape)
print('Arrivals rows:', arrivals.shape)
print('World years:', int(world['year'].min()), int(world['year'].max()))
print('Arrivals years:', int(arrivals['year'].min()), int(arrivals['year'].max()))

World rows: (6650, 11)
Arrivals rows: (5043, 4)
World years: 1999 2023
Arrivals years: 1995 2020


In [16]:
# Merge by ISO3 + year (use overlapping years for model training/evaluation)
merged = world.merge(
    arrivals[['country_code', 'year', 'intl_arrivals']],
    on=['country_code', 'year'],
    how='inner',
)

merged = merged[(merged['year'] >= 1999) & (merged['year'] <= 2020)].copy()
merged = merged.sort_values(['country_code', 'year']).reset_index(drop=True)

merged[['country', 'country_code', 'year', 'intl_arrivals']].head()

,country,country_code,year,intl_arrivals
0,Aruba,ABW,1999,972000.0
1,Aruba,ABW,2000,1211000.0
2,Aruba,ABW,2001,1178000.0
3,Aruba,ABW,2002,1225000.0
4,Aruba,ABW,2003,1184000.0


In [17]:
# Feature engineering
# Economic/tourism ratios
merged['travel_balance'] = merged['tourism_receipts'] - merged['tourism_expenditures']
merged['receipts_per_arrival'] = merged['tourism_receipts'] / merged['tourism_arrivals']
merged['receipts_per_arrival'] = merged['receipts_per_arrival'].replace([np.inf, -np.inf], np.nan)

# Time-series features (per country)
merged['intl_arrivals_lag1'] = merged.groupby('country_code')['intl_arrivals'].shift(1)
merged['intl_arrivals_lag2'] = merged.groupby('country_code')['intl_arrivals'].shift(2)
merged['intl_arrivals_roll3'] = merged.groupby('country_code')['intl_arrivals'].transform(
    lambda s: s.shift(1).rolling(3, min_periods=3).mean()
)

# Target: next-year arrivals
merged['y_next_year'] = merged.groupby('country_code')['intl_arrivals'].shift(-1)

# Keep only rows where target exists (i.e., not the last year in each country series)
df_model = merged.dropna(subset=['y_next_year']).copy()

print('Model rows:', df_model.shape)
df_model[['country', 'country_code', 'year', 'intl_arrivals', 'y_next_year']].head()

Model rows: (3797, 18)


,country,country_code,year,intl_arrivals,y_next_year
0,Aruba,ABW,1999,972000.0,1211000.0
1,Aruba,ABW,2000,1211000.0,1178000.0
2,Aruba,ABW,2001,1178000.0,1225000.0
3,Aruba,ABW,2002,1225000.0,1184000.0
4,Aruba,ABW,2003,1184000.0,1304000.0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Features for forecasting
feature_cols = [
    'intl_arrivals',
    'intl_arrivals_lag1',
    'intl_arrivals_lag2',
    'intl_arrivals_roll3',
    'tourism_receipts',
    'tourism_exports',
    'tourism_departures',
    'tourism_expenditures',
    'gdp',
    'inflation',
    'unemployment',
    'travel_balance',
    'receipts_per_arrival',
]

X = df_model[feature_cols]
y = df_model['y_next_year']

# Time-aware evaluation: train up to 2018, test on 2019 (normal year)
train_mask = df_model['year'] <= 2018
test_mask = df_model['year'] == 2019

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

print('Train rows:', X_train.shape[0], 'Test rows:', X_test.shape[0])

# Baseline: predict next year equals this year's arrivals
baseline_pred = df_model.loc[test_mask, 'intl_arrivals'].values
baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))

ridge_model = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=1.0, random_state=42)),
    ]
)

ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)
ridge_mae = mean_absolute_error(y_test, ridge_pred)
ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))

results = pd.DataFrame([
    {'Model': 'Baseline (y[t])', 'MAE': baseline_mae, 'RMSE': baseline_rmse},
    {'Model': 'Ridge', 'MAE': ridge_mae, 'RMSE': ridge_rmse},
]).sort_values('RMSE')

results

Train rows: 3665 Test rows: 132


TypeError: got an unexpected keyword argument 'squared'

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('model', RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)),
    ]
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

results = pd.concat(
    [
        results,
        pd.DataFrame([{'Model': 'RandomForest', 'MAE': rf_mae, 'RMSE': rf_rmse}]),
    ],
    ignore_index=True,
).sort_values('RMSE')

results

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,2.227051,-1.634768,July,-0.012141,-1.685297,-0.92889,-1.310240,0.247897,...,No Deposit,NaN,NaN,-0.131924,Transient,-2.015038,-0.254873,-0.720694,Check-Out,2015-07-01
1,Resort Hotel,0,5.923385,-1.634768,July,-0.012141,-1.685297,-0.92889,-1.310240,0.247897,...,No Deposit,NaN,NaN,-0.131924,Transient,-2.015038,-0.254873,-0.720694,Check-Out,2015-07-01
2,Resort Hotel,0,-0.907814,-1.634768,July,-0.012141,-1.685297,-0.92889,-0.786207,-1.478447,...,No Deposit,NaN,NaN,-0.131924,Transient,-0.530935,-0.254873,-0.720694,Check-Out,2015-07-02
3,Resort Hotel,0,-0.851667,-1.634768,July,-0.012141,-1.685297,-0.92889,-0.786207,-1.478447,...,No Deposit,1.961711,NaN,-0.131924,Transient,-0.530935,-0.254873,-0.720694,Check-Out,2015-07-02
4,Resort Hotel,0,-0.842309,-1.634768,July,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,...,No Deposit,1.383958,NaN,-0.131924,Transient,-0.075810,-0.254873,0.540666,Check-Out,2015-07-03


In [ ]:
import matplotlib.pyplot as plt

# Plot actual vs predicted for the evaluation year (2019 -> 2020 target)
plot_df = df_model.loc[test_mask, ['country', 'country_code', 'year', 'y_next_year']].copy()
plot_df['pred_ridge'] = ridge_pred
plot_df['pred_rf'] = rf_pred

# Show the countries with largest errors (optional quick diagnostic)
plot_df['abs_err_ridge'] = (plot_df['y_next_year'] - plot_df['pred_ridge']).abs()
plot_df.sort_values('abs_err_ridge', ascending=False).head(10)

In [ ]:
# Train final model on all available training rows up to 2019, then forecast 2021 using 2020 features
train_full = df_model[df_model['year'] <= 2019].copy()
forecast_base = merged[merged['year'] == 2020].copy()

# Recompute features on forecast_base to ensure lag features are present
forecast_base = forecast_base.sort_values(['country_code', 'year'])

# Keep only rows with the lag features available (some countries may have sparse series)
forecast_X = forecast_base[feature_cols]

# Choose best model by RMSE from results table
best_model_name = results.iloc[0]['Model']
if best_model_name == 'RandomForest':
    final_model = rf_model
else:
    final_model = ridge_model

final_model.fit(train_full[feature_cols], train_full['y_next_year'])
forecast_2021 = final_model.predict(forecast_X)

rank_df = forecast_base[['country', 'country_code', 'intl_arrivals']].copy()
rank_df.rename(columns={'intl_arrivals': 'intl_arrivals_2020'}, inplace=True)
rank_df['pred_intl_arrivals_2021'] = forecast_2021
rank_df['pred_growth'] = rank_df['pred_intl_arrivals_2021'] - rank_df['intl_arrivals_2020']

rank_df = rank_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['pred_intl_arrivals_2021'])
rank_df.sort_values('pred_intl_arrivals_2021', ascending=False).head(10)

In [ ]:
# Top destinations by predicted arrivals (2021)
top_by_arrivals = rank_df.sort_values('pred_intl_arrivals_2021', ascending=False).head(15)

# Top emerging destinations by predicted growth (2021 vs 2020)
top_by_growth = rank_df.sort_values('pred_growth', ascending=False).head(15)

print('Top predicted destinations by arrivals (2021):')
display(top_by_arrivals)

print('Top predicted destinations by growth (2021 vs 2020):')
display(top_by_growth)

plt.figure(figsize=(12, 5))
plt.bar(top_by_growth['country_code'], top_by_growth['pred_growth'])
plt.title('Top Predicted Growth in International Arrivals (2021 vs 2020)')
plt.xlabel('Country code')
plt.ylabel('Predicted growth (arrivals)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()